In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install emoji soynlp pytorch_lightning transformers seqeval

In [ ]:
import os 
import random
import torch
import numpy as np
import pandas as pd 
import copy 
import json
import argparse 
import glob 

from scipy.stats import pearsonr, spearmanr
from seqeval import metrics as seqeval_metrics
from sklearn import metrics as sklearn_metrics
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from fastprogress.fastprogress import master_bar, progress_bar
from transformers import (
    AdamW,
    get_linear_schedule_with_warmup
)
from transformers import ( 
    ElectraConfig,
    ElectraForSequenceClassification
)
from torch import nn
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from torch.utils.data import TensorDataset

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/LAMDA/chatbot/dataset/Wellness_Conversation_intent.csv')
data

,intent,keyword(임상키워드),utterance(2차),intent_label
0,우울감,우울,임신해서 우울해,0
1,우울감,NaN,아이 가지고 나서 우울해,0
2,우울감,NaN,아이 가졌는데 기분 하나도 안 좋고 울적해,0
3,우울감,NaN,임신했는데 남편이 하나도 안 챙겨줘서 우울하다,0
4,우울감,NaN,진단 결과 안 좋게 나올 것 같아서 우울해,0
...,...,...,...,...
19672,불안,NaN,그래도 잠못자고 불안한건 여전해요.,18
19673,불안,NaN,불안함에 항상 시달리니까 잠도 못잤어요.,18
19674,불안,NaN,불안하고 초조해서 잠이 안 와.,18
19675,불안,NaN,너무 불안하니까 밤만 되면 잠이 안 오고 너무 초조해.,18


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19677 entries, 0 to 19676
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   intent          19677 non-null  object
 1   keyword(임상키워드)  1976 non-null   object
 2   utterance(2차)   19677 non-null  object
 3   intent_label    19677 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 615.0+ KB


In [ ]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
intent_label,19677.0,8.248361,5.445685,0.0,3.0,9.0,12.0,18.0


In [ ]:
data = data[['intent', 'utterance(2차) ', 'intent_label']]
data.columns = ['intent', 'comment', 'intent_label'] 
data

,intent,comment,intent_label
0,우울감,임신해서 우울해,0
1,우울감,아이 가지고 나서 우울해,0
2,우울감,아이 가졌는데 기분 하나도 안 좋고 울적해,0
3,우울감,임신했는데 남편이 하나도 안 챙겨줘서 우울하다,0
4,우울감,진단 결과 안 좋게 나올 것 같아서 우울해,0
...,...,...,...
19672,불안,그래도 잠못자고 불안한건 여전해요.,18
19673,불안,불안함에 항상 시달리니까 잠도 못잤어요.,18
19674,불안,불안하고 초조해서 잠이 안 와.,18
19675,불안,너무 불안하니까 밤만 되면 잠이 안 오고 너무 초조해.,18


In [ ]:
config = {} 

config['train_file'] = '/content/drive/MyDrive/LAMDA/chatbot/dataset/Wellness_Conversation_intent_train.tsv'   #
config['test_file'] = '/content/drive/MyDrive/LAMDA/chatbot/dataset/Wellness_Conversation_intent_test.tsv' 
config['dev_file'] = ''
config['evaluate_test_during_training'] = True 
config['eval_all_checkpoints'] = True
config['save_optimizer'] = False 
config['do_lower_case'] = False 
config['do_train'] = True 
config['do_eval'] = True 
config['max_grad_norm'] = 1 
config['no_cuda'] = False 
config['model_type'] = "koelectra-base"
config['model_name_or_path'] = "monologg/koelectra-base-discriminator"
config['output_dir'] = "koelectra-base-wellness-ckpt"
config['weight_decay'] = 0 
config['gradient_accumulation_steps'] = 1
config['adam_epsilon'] = 1e-8 
config['warmup_proportion'] = 0
config['num_labels'] = 19 

config['max_seq_len'] = 128 
config['num_train_epochs'] = 50 
config['max_steps'] = -1 
config['seed'] = 42 
config['train_batch_size'] = 32 
config['eval_batch_size'] = 128
config['logging_steps'] = 2000
config['save_steps'] = 2000
config['learning_rate'] = 5e-5 
config

{'train_file': '/content/drive/MyDrive/LAMDA/chatbot/dataset/Wellness_Conversation_intent_train.tsv',
 'test_file': '/content/drive/MyDrive/LAMDA/chatbot/dataset/Wellness_Conversation_intent_test.tsv',
 'dev_file': '',
 'evaluate_test_during_training': True,
 'eval_all_checkpoints': True,
 'save_optimizer': False,
 'do_lower_case': False,
 'do_train': True,
 'do_eval': True,
 'max_grad_norm': 1,
 'no_cuda': False,
 'model_type': 'koelectra-base',
 'model_name_or_path': 'monologg/koelectra-base-discriminator',
 'output_dir': 'koelectra-base-wellness-ckpt',
 'weight_decay': 0,
 'gradient_accumulation_steps': 1,
 'adam_epsilon': 1e-08,
 'warmup_proportion': 0,
 'num_labels': 19,
 'max_seq_len': 128,
 'num_train_epochs': 50,
 'max_steps': -1,
 'seed': 42,
 'train_batch_size': 32,
 'eval_batch_size': 128,
 'logging_steps': 2000,
 'save_steps': 2000,
 'learning_rate': 5e-05}

In [ ]:
class InputExample(object):
    """
    A single training/test example for simple sequence classification.
    """
    def __init__(self, guid, text_a, text_b):
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"

In [ ]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, attention_mask, token_type_ids):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.token_type_ids = token_type_ids

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"

In [ ]:
def get_examples(mode, comments):
  examples = []

  for idx, comment in comments: 
    guid = "%s-%s" % (mode, idx)
    text_a = comment
    examples.append(InputExample(guid=guid, text_a=text_a, text_b=None, label=label))

  return examples

In [ ]:
def convert_examples_to_features(args, examples, tokenizer, max_length, task):
    batch_encoding = tokenizer.batch_encode_plus(
        [(example.text_a, example.text_b) for example in examples],
        max_length=max_length,
        padding="max_length",
        add_special_tokens=True,
        truncation=True,
    )
    
    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}
        if "token_type_ids" not in inputs:
            inputs["token_type_ids"] = [0] * len(inputs["input_ids"])  # For xlm-roberta
        feature = InputFeatures(**inputs)
        features.append(feature)
    return features

In [ ]:
def create_dataset(args, features, tokenizer): 
  all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
  all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
  all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)

  dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids)
  return dataset 

In [ ]:
labels = list(range(19))
labels

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [ ]:
config2 = ElectraConfig.from_pretrained(   # ElectraConfig.from_pretrained
        config['model_name_or_path'],
        num_labels = 19,   # args.task: wellness, num_labels = 19 
        id2label={str(i): label for i, label in enumerate(labels)},   # labels: ['0', '1', '2', ... ,'18']
        label2id={label: i for i, label in enumerate(labels)},
)

# config2

In [ ]:
tokenizer = ElectraTokenizer.from_pretrained(   # model_type: koelectra-base, ElectraTokenizer 
    config['model_name_or_path'],
    do_lower_case = config['do_lower_case'],
)

model = ElectraForSequenceClassification.from_pretrained(
    config['model_name_or_path'],
    config=config2
)

Some weights of the model checkpoint at monologg/koelectra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-discriminator and are newly initialized: ['clas

In [ ]:
config['device'] = "cuda" if torch.cuda.is_available() and not config['no_cuda'] else "cpu"
model.to(config['device'])

In [ ]:
model_path = '/content/drive/MyDrive/LAMDA/chatbot/dataset/'
model_name = 'KoELECTRA_intent.pt'

In [ ]:
model.load_state_dict(torch.load(os.path.join(model_path, model_name)))

<All keys matched successfully>

In [ ]:
model

In [ ]:
data.intent.unique()

array(['우울감', '슬픔', '외로움', '분노', '무기력', '감정조절이상', '상실감', '식욕저하', '식욕증가',
       '불면', '초조함', '피로', '죄책감', '집중력저하', '자신감저하', '자존감저하', '절망감', '자살충동',
       '불안'], dtype=object)

In [ ]:
def get_idx(logits):
  '''
  총합이 1인 Softmax 배열을 입력으로 받아 임계값을 넘는 label idx를 반환하는 함수 
  '''
  threshold = 0.3
  idx_list = [] 
  for idx, prob in enumerate(list(logits)): 
    if prob > threshold: 
      idx_list.append(idx)
  
  return idx_list

In [ ]:
def get_multi_label(args, model, test_dataset):
    results = {}
    test_sampler = SequentialSampler(test_dataset)
    test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=args['eval_batch_size'])

    preds = None
    for batch in progress_bar(test_dataloader):
        model.eval()
        batch = tuple(t.to(args['device']) for t in batch)   # args.device: cuda 

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2], 
            }
            outputs = model(**inputs)
            logits = outputs[:1]  
            # print(logits[0])
            m = torch.nn.functional.softmax(logits[0], dim=-1)
            # print(m)  # [[19개의 확률 값], [19개의 확률 값], ... [19개의 확률 값]]
        if preds is None:   # 초기 
            preds = m.detach().cpu().numpy()   # 예측 확률 
        else:
            preds = np.append(preds, m.detach().cpu().numpy(), axis=0)  
    # print(preds)
    print(get_idx(preds[0]), get_idx(preds[1]), get_idx(preds[2]))      
    return preds

In [ ]:
class testExample(object):
    """
    A single training/test example for simple sequence classification.
    """
    def __init__(self, guid, text_a, text_b):
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"

In [ ]:
comments = ['요즘 너무 불안해서 그런가 잠도 잘 안 오고 우울해', '오늘 학교에서 엄청 안 좋은 일이 있었어, 그리고 지금은 좀 초조해', '왜 이렇게 집중 안되고 슬프지']

In [ ]:
examples = []
for (i, line) in enumerate(comments):
    guid = "%s-%s" % ("test", i)
    text_a = comments[i]
    examples.append(testExample(guid=guid, text_a=text_a, text_b=None))

In [ ]:
examples

[{
   "guid": "test-0",
   "text_a": "\uc694\uc998 \ub108\ubb34 \ubd88\uc548\ud574\uc11c \uadf8\ub7f0\uac00 \uc7a0\ub3c4 \uc798 \uc548 \uc624\uace0 \uc6b0\uc6b8\ud574",
   "text_b": null
 }, {
   "guid": "test-1",
   "text_a": "\uc624\ub298 \ud559\uad50\uc5d0\uc11c \uc5c4\uccad \uc548 \uc88b\uc740 \uc77c\uc774 \uc788\uc5c8\uc5b4, \uadf8\ub9ac\uace0 \uc9c0\uae08\uc740 \uc880 \ucd08\uc870\ud574",
   "text_b": null
 }, {
   "guid": "test-2",
   "text_a": "\uc65c \uc774\ub807\uac8c \uc9d1\uc911 \uc548\ub418\uace0 \uc2ac\ud504\uc9c0",
   "text_b": null
 }]

In [ ]:
features = convert_examples_to_features(
            config, examples, tokenizer, max_length=config['max_seq_len'], task='wellness'
      )

In [ ]:
# Convert to Tensors and build dataset
all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)

test_dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids)
test_dataset

In [ ]:
y_pred = get_multi_label(config, model, test_dataset)

[0, 18] [10] [0, 1]
